In [1]:
import pandas as pd
import re
import pickle
import io

In [2]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [3]:
tokenizer = Tokenizer(num_words=20000,lower=False, filters='!"$%&()*+,-.;<=>?[\\]^_`{|}~\t\n')

In [4]:
taskFile = './results_sarcasmo.csv'

In [5]:
def normalizeText(tweet):
    tweet = re.sub(r'#(S|s)arcasm|#(I|i)rony','',tweet)
    tweet = re.sub(r'#SARCASM|#IRONY','',tweet)
    tweet = re.sub(r'https?://t\.co/.(\w|\d)+','http://link', tweet) #tweet link
    tweet = re.sub(r'@.\w*','@',tweet)
    tweet = re.sub(r'#','',tweet)
    return tweet

def buildTokenizer(tweets):
    tokenizer.fit_on_texts(tweets)
    return tokenizer

def text2seq(tweet):
    return tokenizer.texts_to_sequences([tweet])[0] # Return array like [1 2 3] instead of [[1 2 3]]

In [6]:
def read_ds(ds_list):
    supreme = []
    for tweet in ds_list:
        supreme.append(tweet.split('|',2)[-2:])
    return supreme


In [7]:
with io.open('corpus_50_50.txt','r',encoding='utf8') as f:
    corpus = f.readlines()

In [8]:
corpus = read_ds(corpus)
for i in corpus[:10]:
    print (i)

['1', 'Tengo tantas ganas de comer sopa\n']
['1', '@ @ @ @ faltó el ingenioso @ . 😅\n']
['0', '@ donde sea pero que se le caiga el pipi charlie charlie\n']
['1', '@ Yo ya te sigo, me gustan y tus fotos y el video de hace poco genialisimo  ON\n']
['1', '@ @ Ah bueno si, respuesta si jajajajaja\n']
['1', 'Partidazo de Noir\n']
['1', '@ pero y como es que descubren eso ahora!!\n']
['1', 'Más Pymes, cómo se habrá enterado que todos necesitamos un portasandia en la vida, el negociazo!    http://link\n']
['0', 'Hablar Con El Como Lo Mas Hermoso Que Hay ,El Que Entiende Mis Visajes Y Yo Los De El ♡.♡\n']
['1', 'Hablemos con el Representante. El le responde al pueblo...  http://link\n']


In [9]:
data = pd.DataFrame(data=corpus,columns=['CATEGORY','TWEET'])

In [10]:
data.head()

CATEGORY                                              TWEET
0        1                 Tengo tantas ganas de comer sopa\n
1        1                 @ @ @ @ faltó el ingenioso @ . 😅\n
2        0  @ donde sea pero que se le caiga el pipi charl...
3        1  @ Yo ya te sigo, me gustan y tus fotos y el vi...
4        1         @ @ Ah bueno si, respuesta si jajajajaja\n

In [11]:
token = buildTokenizer(data.TWEET)

In [12]:
token.word_index

{'flamenco': 7080,
 'Jacobo': 17972,
 'insólitas': 21133,
 'concentrarse': 28399,
 'claudio': 16739,
 'solucion': 8172,
 'barquisimetanxs': 14703,
 'informa': 14916,
 'Trejo': 16610,
 'Tienen': 2723,
 'LaBellayLaBestia': 27564,
 'taohun': 24476,
 'Nadal': 6166,
 'Domingo': 1820,
 'pure': 13101,
 'aclara': 19445,
 'ladilla': 24138,
 'marley': 9937,
 'parasito': 23659,
 'mexico': 5742,
 'FFVII': 26663,
 'moda': 1419,
 'א': 18087,
 'preparando': 8810,
 'ton': 27259,
 'maximo': 5930,
 'acostaron': 19575,
 'ocurrencia': 20505,
 'cogiendo': 12265,
 '88': 5535,
 'unirme': 26381,
 'cel': 1747,
 'putitos': 22126,
 'MeDescojonoVivo': 22230,
 'mercaillo': 21908,
 'coherente': 4234,
 'barbie': 21973,
 'comprometidas': 27054,
 'autoinvita': 25451,
 'hanko': 9779,
 'argumentar': 10805,
 'playback': 4905,
 'coloso': 18415,
 'dinerito': 18574,
 'teléfonos': 15974,
 'posee': 27996,
 'aparece': 1742,
 'argentinos': 4814,
 'Benson': 26777,
 'gustaron': 24217,
 'podrias': 6273,
 'Whatsapp': 3055,
 'nMtvHi

In [13]:
token.word_counts

OrderedDict([('Tengo', 72),
             ('tantas', 41),
             ('ganas', 146),
             ('de', 5963),
             ('comer', 40),
             ('sopa', 3),
             ('@', 4939),
             ('faltó', 7),
             ('el', 3052),
             ('ingenioso', 1),
             ('😅', 5),
             ('donde', 270),
             ('sea', 119),
             ('pero', 619),
             ('que', 6335),
             ('se', 1834),
             ('le', 573),
             ('caiga', 7),
             ('pipi', 1),
             ('charlie', 7),
             ('Yo', 208),
             ('ya', 549),
             ('te', 1341),
             ('sigo', 40),
             ('me', 2926),
             ('gustan', 29),
             ('y', 3344),
             ('tus', 93),
             ('fotos', 65),
             ('video', 34),
             ('hace', 282),
             ('poco', 112),
             ('genialisimo', 1),
             ('ON', 9),
             ('Ah', 34),
             ('bueno', 210),
             ('

In [14]:
print (token.word_index['@'])
print (token.word_index['http://link'])

3
11


In [15]:
data['TWEET'] = data['TWEET'].apply(text2seq)

In [16]:
data.head()

CATEGORY                                              TWEET
0        1                      [268, 456, 146, 2, 469, 4487]
1        1               [3, 3, 3, 3, 2263, 8, 9821, 3, 3047]
2        0  [3, 80, 173, 35, 1, 14, 39, 2264, 8, 9822, 226...
3        1  [3, 108, 41, 21, 470, 10, 660, 6, 216, 300, 6,...
4        1                 [3, 3, 559, 107, 29, 579, 29, 418]

In [17]:
pickle.dump(data,open('./tweet_seq.txt','wb'))

In [18]:
pickle.dump(token,open('./token.p','wb'))